In [16]:
import cv2
import mediapipe as mp
import os
import pandas as pd

mp_pose = mp.solutions.pose

def extract_landmarks_from_video(video_path):
    pose = mp_pose.Pose(static_image_mode=False)
    cap = cv2.VideoCapture(video_path)

    data = []
    frames_con_landmarks = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            frames_con_landmarks += 1
            frame_landmarks = []
            for lm in results.pose_landmarks.landmark:
                frame_landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
            data.append(frame_landmarks)

    cap.release()
    pose.close()

    print(f"📹 {os.path.basename(video_path)} → {frames_con_landmarks} frames con landmarks.")
    return pd.DataFrame(data)

def process_all_videos(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".mp4"):
                full_path = os.path.join(root, file)
                activity = os.path.basename(root)
                output_path = os.path.join(output_dir, f"{activity}_{file.replace('.mp4', '.csv')}")
                
                print(f"🚀 Procesando: {file}")
                df = extract_landmarks_from_video(full_path)

                if not df.empty:
                    df.to_csv(output_path, index=False)
                    print(f"✅ Guardado en: {output_path}")
                else:
                    print(f"⚠️ No se detectaron landmarks en {file}, CSV no generado.")

# Ejecutar
process_all_videos("c:/Users/CTecn/Desktop/Inteligencia-Artificial/data/raw_videos", "c:/Users/CTecn/Desktop/Inteligencia-Artificial/data/processed_landmarks")


🚀 Procesando: user1_espalda_back_1.mp4
📹 user1_espalda_back_1.mp4 → 64 frames con landmarks.
✅ Guardado en: c:/Users/CTecn/Desktop/Inteligencia-Artificial/data/processed_landmarks\caminar_espalda_user1_espalda_back_1.csv
🚀 Procesando: user1_espalda_back_2.mp4
📹 user1_espalda_back_1.mp4 → 64 frames con landmarks.
✅ Guardado en: c:/Users/CTecn/Desktop/Inteligencia-Artificial/data/processed_landmarks\caminar_espalda_user1_espalda_back_1.csv
🚀 Procesando: user1_espalda_back_2.mp4
📹 user1_espalda_back_2.mp4 → 79 frames con landmarks.
✅ Guardado en: c:/Users/CTecn/Desktop/Inteligencia-Artificial/data/processed_landmarks\caminar_espalda_user1_espalda_back_2.csv
🚀 Procesando: user1_espalda_back_3.mp4
📹 user1_espalda_back_2.mp4 → 79 frames con landmarks.
✅ Guardado en: c:/Users/CTecn/Desktop/Inteligencia-Artificial/data/processed_landmarks\caminar_espalda_user1_espalda_back_2.csv
🚀 Procesando: user1_espalda_back_3.mp4
📹 user1_espalda_back_3.mp4 → 72 frames con landmarks.
✅ Guardado en: c:/Users

In [13]:
# Mostrar el directorio de trabajo actual para depuración
import os
print('Directorio de trabajo actual:', os.getcwd())

Directorio de trabajo actual: c:\Users\CTecn\Desktop\Inteligencia-Artificial\notebooks
